# tfp-multiple-changepoint-detection

Author: Henry Cooksley

Adapted from [Multiple changepoint detection and Bayesian model selection](https://www.tensorflow.org/probability/examples/Multiple_changepoint_detection_and_Bayesian_model_selection)

In [ ]:
!pip install -r requirements.txt

In [ ]:
import numpy as np
import tensorflow.compat.v2 as tf
tf.enable_v2_behavior()
import tensorflow_probability as tfp
from tensorflow_probability import distributions as tfd

from matplotlib import pylab as plt
%matplotlib inline
import scipy.stats

In [ ]:
# true_rates = [40, 3, 20, 50]
# true_durations = [10, 20, 5, 35]
true_rates = [50, 3, 20, 50]
true_durations = [35, 20, 5, 35]
observed_counts = np.concatenate([
    scipy.stats.poisson(rate).rvs(num_steps) 
        for (rate, num_steps) in zip(true_rates, true_durations)
]).astype(np.float32)
observed_counts

In [ ]:
plt.plot(observed_counts);

In [ ]:
num_states = 4
initial_state_logits = np.zeros([num_states], dtype=np.float32)
initial_state_logits

In [ ]:
daily_change_prob = 0.05
transition_probs = daily_change_prob / (num_states-1) * np.ones(
    [num_states, num_states], dtype=np.float32)
transition_probs

In [ ]:
np.fill_diagonal(transition_probs, 
                 1-daily_change_prob)

In [ ]:
print(f"Initial state logits:\n{initial_state_logits}")
print(f"Transition matrix:\n{transition_probs}")

In [ ]:
trainable_log_rates = tf.Variable(
    np.log(np.mean(observed_counts)) + tf.random.normal([num_states]),
    name='log_rates')
trainable_log_rates

In [ ]:
hmm = tfd.HiddenMarkovModel(
    initial_distribution=tfd.Categorical(
        logits=initial_state_logits),
    transition_distribution=tfd.Categorical(probs=transition_probs),
    observation_distribution=tfd.Poisson(log_rate=trainable_log_rates),
    num_steps=len(observed_counts))
hmm

In [ ]:
rate_prior = tfd.LogNormal(5, 5)
rate_prior

In [ ]:
def log_prob():
    return (tf.reduce_sum(rate_prior.log_prob(tf.math.exp(trainable_log_rates))) +
           hmm.log_prob(observed_counts))

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.1)
optimizer

In [ ]:
@tf.function(autograph=False)
def train_op():
    with tf.GradientTape() as tape:
        neg_log_prob = -log_prob()
    grads = tape.gradient(neg_log_prob, [trainable_log_rates])[0]
    optimizer.apply_gradients([(grads, trainable_log_rates)])
    return neg_log_prob, tf.math.exp(trainable_log_rates)


In [ ]:
for step in range(201):
    loss, rates = [t.numpy() for t in train_op()]
    if step % 20 == 0:
        print(f"step {step}: log prob {-loss} rates {rates}")

print("\n")
print(f"Inferred rates: {rates}")
print("\n")
print(f"True rates: {true_rates}")

In [ ]:
posterior_dists = hmm.posterior_marginals(observed_counts)
posterior_dists

In [ ]:
posterior_probs = posterior_dists.probs_parameter().numpy()
posterior_probs

In [ ]:
def plot_state_posterior(ax, state_posterior_probs, title):
    ln1 = ax.plot(state_posterior_probs, c='blue', lw=3, label='p(state | counts)')
    ax.set_ylim(0., 1.1)
    ax.set_ylabel('posterior probability')
    ax2 = ax.twinx()
    ln2 = ax2.plot(observed_counts, c='black', alpha=0.3, label='observed counts')
    ax2.set_title(title)
    ax2.set_xlabel('time')
    lns = ln1+ln2
    labs = [l.get_label() for l in lns]
    ax.legend(lns, labs, loc=4)
    ax.grid(True, color='white')
    ax2.grid(False)


In [ ]:
fig = plt.figure(figsize=(10, 10))
nrows_ncols = (2, 2)
plot_state_posterior(fig.add_subplot(*nrows_ncols, 1),
                    posterior_probs[:, 0],
                    title=f"state 0 (rate {rates[0]:.2f})")
plot_state_posterior(fig.add_subplot(*nrows_ncols, 2),
                    posterior_probs[:, 1],
                    title=f"state 1 (rate {rates[1]:.2f})")
plot_state_posterior(fig.add_subplot(*nrows_ncols, 3),
                    posterior_probs[:, 2],
                    title=f"state 3 (rate {rates[2]:.2f})")
plot_state_posterior(fig.add_subplot(*nrows_ncols, 4),
                    posterior_probs[:, 3],
                    title=f"state 4 (rate {rates[3]:.2f})")
plt.tight_layout()

In [ ]:
most_probable_states = np.argmax(posterior_probs, axis=1)
most_probable_states

In [ ]:
most_probable_rates = rates[most_probable_states]
most_probable_rates

In [ ]:
fig = plt.figure(figsize=(10, 4))
ax = fig.add_subplot(1, 1, 1)
ax.plot(most_probable_rates, c='green', lw=3, label='inferred rate')
ax.plot(observed_counts, c='black', alpha=0.3, label='observed_counts')
ax.set_ylabel('latent_rate')
ax.set_xlabel('time')
ax.set_title('Inferred latent rate over time')
ax.legend(loc=4);